# Career Q&A Chatbot - Model Training

In [1]:
# Install dependencies
!pip install transformers tensorflow pandas scikit-learn sacrebleu gradio torch

  Using cached torch-2.6.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.6.0-cp311-cp311-win_amd64.whl (204.2 MB)
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   ------ --------------------------------- 1.0/6.2 MB 2.0 MB/s eta 0:00:03
   ---------- ----------------------------- 1.6/6.2 MB 2.3 MB/s eta 0:00:03
   ----------- ---------------------------- 1.8/6.2 MB 2.4 MB/s eta 0:00:02
   ---------------- ----------------------- 2.6/6.2 MB 2.5 MB/s eta 0:00:02
   -------------------- ------------------- 3.1/6.2 MB 2.4 MB/s eta 0:00:02
   -------------------


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from sklearn.model_selection import train_test_split

c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load dataset
df = pd.read_csv("Data/CareerDataset.csv")
print("Dataset columns:", df.columns)
print("\nSample data:\n", df.head())

Dataset columns: Index(['role', 'question', 'answer'], dtype='object')

Sample data:
              role                                           question  \
0  Data Scientist                     What does a Data Scientist do?   
1  Data Scientist  What are the main responsibilities of a Data S...   
2  Data Scientist  What is the job description for a Data Scientist?   
3  Data Scientist  What skills are required to become a Data Scie...   
4  Data Scientist  What are the essential skills for a successful...   

                                              answer  
0  A Data Scientist extracts meaningful insights ...  
1  Responsibilities include data cleaning, analyz...  
2  A Data Scientist is responsible for collecting...  
3  Skills required include expertise in Python or...  
4  A successful Data Scientist needs strong analy...  


In [4]:
df = df[["question", "answer"]]

# Add prefix for T5 task conditioning
df["input_text"] = "answer career question: " + df["question"]
df["target_text"] = df["answer"]

# Split data
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [5]:
# Tokenization function (fixed for tuples)
def tokenize_data(input_text_tensor, target_text_tensor):
    input_text = input_text_tensor.numpy().decode("utf-8")
    target_text = target_text_tensor.numpy().decode("utf-8")
    
    inputs = tokenizer(
        input_text,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    targets = tokenizer(
        target_text,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )
    return (
        inputs["input_ids"].squeeze(), 
        inputs["attention_mask"].squeeze(), 
        targets["input_ids"].squeeze()
    )

# Map wrapper for tuples
def map_wrapper(input_text, target_text):
    return tf.py_function(
        tokenize_data,
        [input_text, target_text],
        (tf.int32, tf.int32, tf.int32)
    )

# Convert tuples to dict
def to_dict(input_ids, attention_mask, labels):
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# Dataset pipelines
train_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (train_df["input_text"], train_df["target_text"])
    )
    .map(map_wrapper)
    .map(to_dict)
    .shuffle(1000)
    .batch(8)
    .prefetch(tf.data.AUTOTUNE)
)

val_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (val_df["input_text"], val_df["target_text"])
    )
    .map(map_wrapper)
    .map(to_dict)
    .batch(8)
    .prefetch(tf.data.AUTOTUNE)
)

In [6]:
def map_wrapper(input_text, target_text):
    # Use tf.py_function with output signature as a TUPLE of types
    return tf.py_function(
        tokenize_data,
        [input_text, target_text],
        (tf.int32, tf.int32, tf.int32)  # Output types for input_ids, attention_mask, labels
    )

# Convert tuples to dictionaries in the dataset pipeline
def to_dict(input_ids, attention_mask, labels):
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

# Create datasets
# Create datasets with TUPLE structure (input_text, target_text)
train_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (train_df["input_text"], train_df["target_text"])
    )
    .map(map_wrapper)
    .batch(8)
    .prefetch(tf.data.AUTOTUNE)
)

val_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (val_df["input_text"], val_df["target_text"])
    )
    .map(map_wrapper)
    .batch(8)
    .prefetch(tf.data.AUTOTUNE)
)


In [7]:
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

# Load model and tokenizer (fixed)
model = TFT5ForConditionalGeneration.from_pretrained("t5-small", from_pt=True)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer)

# Train
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Epoch 1/3


OperatorNotAllowedInGraphError: in user code:

    File "c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\modeling_tf_utils.py", line 1658, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    Oper